# AOC 2022

In [220]:
\cd notebooks

'io



# Advent of Code 2022 - 09

URL: https://adventofcode.com/2022/day/9

We'll study ngn's [solution](https://codeberg.org/ngn/k/src/branch/master/aoc/22/09.k):

For this problem, there's a sort of simulation to do.

In [10]:
x:+\'0,'(+,/-:\=2)@\:"DRUL"?*'(`I$2_')#0:"i/09"
f:{x+s*~y~s:-1|1&y-:x}\ /follow
`0:$(#?f/[;+x]@)'1 9 /both parts

6642
2765


Input lines look like the following

In [11]:
5#0:"i/09"

("R 1"
 "U 2"
 "R 1"
 "L 2"
 "D 2")


First part of transforming the input uses `filter` with a monad. As we learned previously, `filter` can get a predicate function. Remember that the predicate gets the full list, and it returns the counts that we get from the filter, so we not just return true/false but 0,1,2.. to get replications. For example: 

In [54]:
{x}#!5

1 2 2 3 3 3 4 4 4 4


In this case, the filter is ```(`I$2_')```, which takes the number, and replicates the line that many times.  After that, we get only the initial letter.

In [77]:
:i2:*'(`I$2_')#30#0:"i/09"
"DRUL"?i2

"RUURLLDDURRULLUDDRDLUUDDLLDUUDDLULLDDUULUULRRU"
1 2 2 1 3 3 0 0 2 1 1 2 3 3 2 0 0 1 0 3 2 2 0 0 3 3 0 2 2 0 0 3 2 3 3 0 0 2 2 3 2 2 3 1 1 2


From the movement list, we convert them to coordinates.

`each-left` of `at` from 

`(+,/-:\=2)@\`. Let's unpack this one

In [80]:
=2
-:\=2
,/-:\=2
(+,/-:\=2)

(1 0
 0 1)
((1 0;0 1)
 (-1 0;0 -1))
(1 0
 0 1
 -1 0
 0 -1)
(1 0 -1 0
 0 1 0 -1)


In [86]:
(+,/-:\=2)@\:"DRUL"?*'(`I$2_')#30#0:"i/09"

(0 -1 -1 0 0 0 1 1 -1 0 0 -1 0 0 -1 1 1 0 1 0 -1 -1 1 1 0 0 1 -1 -1 1 1 0 -1 0 0 1 1 -1 -1 0 -1 -1 0 0 0 -1
 1 0 0 1 -1 -1 0 0 0 1 1 0 -1 -1 0 0 0 1 0 -1 0 0 0 0 -1 -1 0 0 0 0 0 -1 0 -1 -1 0 0 0 0 -1 0 0 -1 1 1 0)


In [21]:
x:+\'0,'(+,/-:\=2)@\:"DRUL"?*'(`I$2_')#0:"i/09"

Let's see the shape of it, to better understand what we did here:

In [89]:
10#'x

(0 0 -1 -2 -2 -2 -2 -1 0 -1
 0 1 1 1 2 1 0 0 0 0)


These are the positions of the Head for each movement. We get to this with a sum over the discrete movements, applied to an initial `0,0` position.

The `follow` function does calculate the difference between the head and the tail, then it bounds the diffs to +1 and -1, because at most, the tail can move one position.

After applying the bounding box of -1,-1...1,1, we do a test to see if we have to apply any movement or not. We do this by multiplying the movements by 0 or 1, and summing that to the current position of the tail.

In [23]:
f:{x+s*~y~s:-1|1&y-:x}\ /follow

In [8]:
f:{x+s*~y~s:-1|1&y-:x}\ 
f[(1 1; -1 -2)] / from 1 1 to -1 -2, the tail moves to 0 0.

(1 1
 0 0)


Now, the tour de force of this exercise:

In [39]:
(#?f/[;+x]@)'1 

6642


Intuitively it's clear what it's doing: for both size 1 and 9, count the unique coordinates (`#?`) from the result of applying `follow`.  We're going to see how to get there.


`(#?f/[;+x]@)` is a train. Note that the `@` is used to make the whole parethesised chunk a train. `f/[;+x]` is as a `noun`, so we need a verb at the rightmost position of the train to make it really a train. That's why we use `@`. 

`f/[;+x]` is a projection, or a partially applied function. K parsing binds `f/[]` together, so `@` applies the argument to the whole `f/[;+x]`. The unbound parameter is the first argument to `f/[]`, and `f` being a monad, the first parameter will be the number of iterations.

At this point, it might not be obvious why this incantation works, and how the lenght of the rope can be related to the number of times we iterate through `follow`.



## Combining`/`s and `\`

Here are some tests to clear how \ and / work when combined, and how they behave with monads or dyads.

The most basic case is a nested `over` until a fixedpoint is reached.

In [51]:
flatten:,// 
flatten 2 2 2 2#!100

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15


To debug further, we can see the intermediate results by changing the outer `/` to `\`. This doesn't change the computation. If we want the final result to be the same, we should pick the last item of the list, but that's all

In [27]:
,/\2 2 2 2#!100

((((0 1;2 3);(4 5;6 7));((8 9;10 11);(12 13;14 15)))
 ((0 1;2 3);(4 5;6 7);(8 9;10 11);(12 13;14 15))
 (0 1;2 3;4 5;6 7;8 9;10 11;12 13;14 15)
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15)


The inner one can't be changed so easily, because that does change what we are computing.   So, we'll use `\` instead of `/` in our tests here

In [7]:
m:{x}
d:{x+y}
l:!10
m/l  /monad, 1 iteration till fixedpoint
2m/l  /monad, with  iteration count

d\l      /dyad. 1 pass through the list
10 d\l   /dyad with seed. (gets passed as the first `x`)
d\[10;l] /dyad, used in the m-expression form

d\/[0;l]

0 1 2 3 4 5 6 7 8 9
0 1 2 3 4 5 6 7 8 9
0 1 3 6 10 15 21 28 36 45
10 11 13 16 20 25 31 38 46 55
10 11 13 16 20 25 31 38 46 55
0 1 2 3 4 5 6 7 8 9


For `d\/l`, think that `d\` makes a monad.  So, to the rightmost `/` we should pass a number of iterations we want to run, or it might run indefinitely until finding a fixedpoint.

In [34]:
d\/[0;l]
d\/[1;l]
/ If we change the `\/` to `\\`, we will get a trail of all the partial results 
d\\[1;l] /running sum
d\\[3;l] /running sum of the running sum of the running sum 
/ of course, it can get done with nested lists
:ll:+!2 2
d\ll
d\\[1;ll]
d\\[3;ll]

0 1 2 3 4 5 6 7 8 9
0 1 3 6 10 15 21 28 36 45
(0 1 2 3 4 5 6 7 8 9
 0 1 3 6 10 15 21 28 36 45)
(0 1 2 3 4 5 6 7 8 9
 0 1 3 6 10 15 21 28 36 45
 0 1 4 10 20 35 56 84 120 165
 0 1 5 15 35 70 126 210 330 495)
(0 0
 0 1
 1 0
 1 1)
(0 0
 0 1
 1 1
 2 2)
((0 0;0 1;1 0;1 1)
 (0 0;0 1;1 1;2 2))
((0 0;0 1;1 0;1 1)
 (0 0;0 1;1 1;2 2)
 (0 0;0 1;1 2;3 4)
 (0 0;0 1;1 3;4 7))


In [51]:
+\\[;10#1]4 / all ones, with accumulated running sums

(1 1 1 1 1 1 1 1 1 1
 1 2 3 4 5 6 7 8 9 10
 1 3 6 10 15 21 28 36 45 55
 1 4 10 20 35 56 84 120 165 220
 1 5 15 35 70 126 210 330 495 715)


# Back to the challenge

With the new acquired knowledge, we see that different rope lengths are solved by iterating over previous results and apply the same `follow` function over it.

We can see the results of multiple rope sizes:

In [52]:
(f/[;10#+x]@)'0 1 2 3


((0 0;0 1;-1 1;-2 1;-2 2;-2 1;-2 0;-1 0;0 0;-1 0)
 (0 0;0 0;0 0;-1 1;-1 1;-1 1;-1 1;-1 1;-1 1;-1 1)
 (0 0;0 0;0 0;0 0;0 0;0 0;0 0;0 0;0 0;0 0)
 (0 0;0 0;0 0;0 0;0 0;0 0;0 0;0 0;0 0;0 0))


`d\/[2;l]` is going to do reduce `d` across `l`. and later, reduce `d` again across the result of the previous iteration.


If we see `l` as a function of time, `l` (which is the same as `0d\l`) is the places where the head goes through. It's like having a rope of size 0.

The first iteration of `d\l` carries the position of the tail of a rope of size 1.
The first iteration of `d\ d\l` carries the position of the tail of a rope of size 2. Taking the position of the tail of the previous simulation as if it would be the head of another rope of size 1.

My head still hurts. 